In [1]:
from glob import glob
from tqdm import tqdm
import json
import re
import os

In [2]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [3]:
instructions = []
with open('evol-codealpaca-v1.jsonl') as fopen:
    for l in fopen:
        instructions.append(json.loads(l)['instruction'])
len(instructions)

111272

In [4]:
data = []
for i in range(len(instructions)):
    filename = f'evol-codealpaca-v1-chatgpt4/{i}.json'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        d = json.load(fopen)
        
    d['instruction_en'] = instructions[i]
    data.append(d)

In [5]:
len(data)

43503

In [6]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [7]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [8]:
count, indon_count = 0, 0

indon_texts = []
for d in tqdm(data):

    if d['output'] is None:
        continue
        
    if any([r in d['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(d['instruction'])
    indon_texts.append(d['output'])

    count += 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 43503/43503 [00:00<00:00, 212000.51it/s]


In [9]:
len(indon_texts)

86680

In [10]:
print(indon_texts[1])

Untuk mengubah skrip ini, kita Akkan menggunakan konsep pengulangan 'while' dan memastikan index tidak melebihi panjang senarai. Juga, kita harus memastikan skrip kita menangani senarai kosong dengan betul. 

Berikut adalah cara mengubahnya:

```python
  # Menetapkan senarai integer
  arr = [1, 2, 3, 4]

  # Tentukan panjang senarai
  n = len(arr)

  # Pastikan senarai tidak kosong
  if n > 0:
    i = 0
    # Melalui senarai dan keluarkan setiap elemen individu
    while i < n:
        print(arr[i])
        i += 1
  else:
    print("Senarai adalah kosong.")
```

Dalam skrip ini, sebelum kita mulai mengulang, kita memeriksa apakah `n`, yang merupakan panjang senarai, lebih besar dari 0. Jika tidak (yang berarti senarai kosong), kita mencetak pesan yang sesuai. 

Kemudian, kami menggunakan pengulangan 'while' untuk mengulang selama indeks `i` kurang dari panjang `n` senarai. Selama setiap iterasi, kita mencetak elemen senarai dan tambahkan 1 ke indeks, sehingga kita dapat bergerak ke ele

In [ ]:
with open('codealpaca-v1-chatgpt4.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [ ]:
len(set(indon_texts))

In [ ]:
!cp codealpaca-v1-chatgpt4.texts ~/ssd3/ctranslate2

In [11]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/codealpaca-v1-chatgpt4.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

86657

In [12]:
data[0]

{'instruction': "Harap ubah skrip Python berikut agar ia memasukkan pengulangan 'while' daripada pengulangan 'for' yang sedia ada, yang meneruskan melalui item-item dalam senarai integer. \n\nSkrip semasa mempunyai bug di mana ia cuba mencetak objek yang berada di luar batas senarai. Betulkan ralat ini dan ubah skrip untuk menggunakan 'while' bukannya pengulangan 'for'. Pastikan skrip anda menangani senarai kosong dengan betul. \n\n```python\n  # Menetapkan senarai integer\n  arr = [1, 2, 3, 4]\n\n  # Tentukan panjang senarai\n  n = len(arr)\n\n  # Melalui senarai dan keluarkan setiap elemen individu\n  for i in range(n+1):\n      print(arr[i])\n```",
 'output': 'Untuk mengubah skrip ini, kita Akkan menggunakan konsep pengulangan \'while\' dan memastikan index tidak melebihi panjang senarai. Juga, kita harus memastikan skrip kita menangani senarai kosong dengan betul. \n\nBerikut adalah cara mengubahnya:\n\n```python\n  # Menetapkan senarai integer\n  arr = [1, 2, 3, 4]\n\n  # Tentukan

In [18]:
rejected = 0

with open('synthetic-codealpaca-v1-chatgpt4.jsonl', 'w') as fopen_l:
    for d in tqdm(data):
            
        if d['output'] is None:
            continue
        
        output_ms = mapping.get(d['output'])
        instruction_ms = mapping.get(d['instruction'])
        
        indon_ins = False
        if check_indon(d['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(d['output']):
            indon_output = True
        
        rejected_ins = False
            
        if any([r in d['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
            
        if rejected_ins or rejected_output:
            rejected += 1
            
        d['indon_ins'] = indon_ins
        d['indon_output'] = indon_output
        d['instruction_ms'] = instruction_ms
        d['output_ms'] = output_ms
        d['rejected_ins'] = rejected_ins
        d['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(d)}\n')
        count += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 43503/43503 [00:03<00:00, 12748.67it/s]


In [19]:
d

{'instruction': "Semasa menggunakan Amazon Redshift untuk keperluan gudang data saya, saya menghadapi 'DiskFullError'. Bolehkah anda memberi penerangan mengenai kaedah-kaedah berkesan untuk meningkatkan dan mengoptimumkan sumber Redshift saya, terutamanya untuk mengendalikan volume data yang besar, sambil pada masa yang sama menyediakan analisis terperinci mengenai asas logik yang kompleks di sebalik pendekatan-pendekatan ini?",
 'output': "Apabila anda menghadapi 'DiskFullError' di Amazon Redshift, ini bermakna kapasiti penyimpanan cluster anda sudah penuh. Berikut adalah beberapa kaedah untuk mengoptimumkan penggunaan sumber di Redshift, yang boleh membantu menangani volume data yang besar:\n\n1. **Tinjauan Kapasiti Penyimpanan:**\n   - Pertama, lakukan tinjauan penggunaan disk dengan memeriksa metric di konsol Redshift atau menggunakan query sistem untuk mengenal pasti penggunaan ruang oleh tabel dan skema.\n\n2. **Pembersihan Data:**\n   - Buang data yang tidak perlu atau kurang pe

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='synthetic-codealpaca-v1-chatgpt4.jsonl',
    path_in_repo='synthetic-codealpaca-v1-chatgpt4.jsonl',
    repo_id='mesolitica/chatgpt4-code-instruct',
    repo_type='dataset',
)

synthetic-codealpaca-v1-chatgpt4.jsonl:   0%|          | 0.00/274M [00:00<?, ?B/s]